In [1]:
# imports

import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import numpy as np
import os
import time

# NBA Data Explorer Class

### Overview

The NBADataExplorer class provides methods for exploring and analyzing NBA play by play data from a SQLite database. This class is specifically designed for injury prediction feature engineering and handles the complex relationships between games, players, and play by play events

### Key Features

**Database Connection & Schema Exploration**
- **connect()** - Establishes connection to NBA SQLite database
- **get_table_info()** - Provides overview of all database tables with record counts
- **explore_table_schemas()** - Deep dive into column structures of core tables

**Data Sampling & Quality Assessment**
- **explore_play_by_play_sample()** - Retrieves play by play data with game context via JOIN operations
- **get_clean_player_actions()** - Filters out technical events and invalid player records
- **test_play_by_play_join_performance()** - Benchmarks query performance for larges cale operations

**Player Analysis & Selection**
- **identify_high_usage_players()** - Finds players with substantial game participation for modeling
- **identify_modeling_candidates()** - Selects players meeting minimum games/actions thresholds
- **analyze_2015_plus_suitability()** - Evaluates player consistency across modern NBA era

**Feature Engineering Foundations**
- **get_player_game_aggregation()** - Aggregates play by play events into player game statistics
- **create_performance_decline_targets()** - Generates injury proxy variables from performance patterns
- **analyze_missing_games()** - Identifies potential injury periods through game absence gaps

**Data Coverage Analysis**
- **get_date_range_analysis()** - Analyzes temporal coverage and data distribution
- **get_modern_era_data()** - Focuses analysis on consistent modern NBA period (2015+)
- **get_yearly_breakdown_2015_plus()** - Year by year data volume breakdown

**EDA Reporting**
- **generate_initial_report()** - Creates complete EDA summary with key metrics and findings

### Strategic Choices Made

**Specific Era Focus (2015+)**
- **Load management era** - Start of league wide and mainstream implementation of load management methods and protocols  
- **Analytics revolution** - More consistent tracking and statistical methodologies. All areans implemented cameras that track players, practices, games, etc.
- **GSW (2015-2016) Effect** - 73 win season arguably altered pace and style league wide 

**Performance Optimization**
- **Selective sampling** - Limits for exploration while maintaining representativeness
- **JOIN performance testing** - Ensures scalability for feature engineering pipeline
- **Clean data filtering** - Removes technical events and invalid records upfront

**Injury Prediction Preparation**
- **Gap analysis** - Identifies absence periods as injury proxies
- **Player consistency** - Ensures sufficient data for reliable modeling
- **Rolling metrics foundation** - Enables workload and fatigue indicator creation

In [2]:
class NBADataExplorer:
    def __init__(self, db_path='../data/raw/nba.sqlite'):
        self.db_path = os.path.join('..', 'data', 'raw', 'nba.sqlite')
        self.conn = None
        
    def connect(self):
        """
        Connects to SQLite database
        """
        self.conn = sqlite3.connect(self.db_path)
        print(f"Connected to {self.db_path}")
        
    def get_table_info(self):
        """
        Gets overview of all tables in database
        """
        query = """
        SELECT name FROM sqlite_master 
        WHERE type='table' 
        ORDER BY name;
        """
        tables = pd.read_sql_query(query, self.conn)
        
        table_info = {}
        for table in tables['name']:
            count_query = f"SELECT COUNT(*) as count FROM {table}"
            count = pd.read_sql_query(count_query, self.conn)['count'].iloc[0]
            table_info[table] = count
            
        return table_info
    
    def explore_table_schemas(self):
        """
        Explores the actual column structure of key tables
        """
        tables_to_explore = ['play_by_play', 'game', 'player', 'common_player_info']
        
        for table in tables_to_explore:
            print(f"\n{table.upper()} Table Schema:")
            schema_query = f"PRAGMA table_info({table})"
            schema_df = pd.read_sql_query(schema_query, self.conn)
            print(schema_df.to_string(index=False))
            
            print(f"\nSample data from {table}:")
            sample_query = f"SELECT * FROM {table} LIMIT 3"
            sample_df = pd.read_sql_query(sample_query, self.conn)
            print(sample_df.head())

    def explore_play_by_play_sample(self, limit=1000):
        """
        Sample play-by-play data WITH dates (requires JOIN w/ game table)
        """
        query = f"""
        SELECT 
            pbp.*,
            g.game_date,
            g.season_id,
            g.season_type
        FROM play_by_play pbp
        JOIN game g ON pbp.game_id = g.game_id
        WHERE g.game_date >= '2020-01-01'
        LIMIT {limit}
        """
        sample_data = pd.read_sql_query(query, self.conn)
        return sample_data
    
    def get_player_game_aggregation(self, player_id=None, limit=100):
        """
        Creates player-game level stats from play-by-play (w/ dates from game table)
        """
        where_clause = f"AND pbp.player1_id = '{player_id}'" if player_id else ""
        
        query = f"""
        SELECT 
            pbp.game_id,
            g.game_date,
            pbp.player1_id as player_id,
            pbp.player1_name as player_name,
            COUNT(*) as total_actions,
            SUM(CASE WHEN pbp.eventmsgtype = 1 THEN 1 ELSE 0 END) as field_goal_attempts,
            SUM(CASE WHEN pbp.eventmsgtype = 4 THEN 1 ELSE 0 END) as rebounds,
            SUM(CASE WHEN pbp.eventmsgtype = 5 THEN 1 ELSE 0 END) as turnovers,
            SUM(CASE WHEN pbp.eventmsgtype = 6 THEN 1 ELSE 0 END) as fouls
        FROM play_by_play pbp
        JOIN game g ON pbp.game_id = g.game_id
        WHERE pbp.player1_id IS NOT NULL
        AND g.game_date >= '2020-01-01'
        {where_clause}
        GROUP BY pbp.game_id, pbp.player1_id, pbp.player1_name, g.game_date
        ORDER BY g.game_date DESC
        LIMIT {limit}
        """
        
        player_stats = pd.read_sql_query(query, self.conn)
        return player_stats
    
    def identify_high_usage_players(self, min_games=20):
        """
        Finds players w/ high game frequency for workload analysis
        """
        query = f"""
        SELECT 
            pbp.player1_id,
            pbp.player1_name,
            COUNT(DISTINCT pbp.game_id) as games_played,
            COUNT(*) as total_actions,
            CAST(COUNT(*) AS FLOAT) / COUNT(DISTINCT pbp.game_id) as avg_actions_per_game
        FROM play_by_play pbp
        JOIN game g ON pbp.game_id = g.game_id
        WHERE pbp.player1_id IS NOT NULL
        AND g.game_date >= '2020-01-01'
        GROUP BY pbp.player1_id, pbp.player1_name
        HAVING COUNT(DISTINCT pbp.game_id) >= {min_games}
        ORDER BY total_actions DESC
        LIMIT 50
        """
        
        high_usage = pd.read_sql_query(query, self.conn)
        return high_usage

    def get_date_range_analysis(self):
        """
        Analyzes the date range and data distribution across years
        """
        query = """
        SELECT 
            MIN(game_date) as earliest_date,
            MAX(game_date) as latest_date,
            COUNT(*) as total_games,
            COUNT(DISTINCT season_id) as total_seasons
        FROM game
        """
        date_range = pd.read_sql_query(query, self.conn)
        
        # Games per year analysis
        games_per_year_query = """
        SELECT 
            SUBSTR(game_date, 1, 4) as year,
            COUNT(*) as games_count
        FROM game
        GROUP BY SUBSTR(game_date, 1, 4)
        ORDER BY year DESC
        LIMIT 10
        """
        games_per_year = pd.read_sql_query(games_per_year_query, self.conn)
        
        return date_range, games_per_year

    def test_play_by_play_join_performance(self):
        """
        Tests how long it takes to query play-by-play with game joins
        """
        
        print("Testing play-by-play JOIN performance...")
        
        # Small test query
        start_time = time.time()
        test_query = """
        SELECT COUNT(*) as count
        FROM play_by_play pbp
        JOIN game g ON pbp.game_id = g.game_id
        WHERE g.game_date >= '2023-01-01'
        """
        result = pd.read_sql_query(test_query, self.conn)
        end_time = time.time()
        
        print(f"JOIN test completed in {end_time - start_time:.2f} seconds")
        print(f"- Found {result['count'].iloc[0]:,} play-by-play records for 2023+")
        
        return result
    
    def get_clean_player_actions(self, start_date='2020-01-01', limit=1000):
        """
        Gets play-by-play data excluding technical/official events
        """
        query = f"""
        SELECT 
            pbp.*,
            g.game_date,
            g.season_id
        FROM play_by_play pbp
        JOIN game g ON pbp.game_id = g.game_id
        WHERE g.game_date >= '{start_date}'
        AND pbp.player1_id IS NOT NULL 
        AND pbp.player1_id != '0'        -- Excludes technical events
        AND pbp.player1_name IS NOT NULL
        AND pbp.player1_name != 'None'   -- Double filter
        LIMIT {limit}
        """
        return pd.read_sql_query(query, self.conn)
    
    def create_performance_decline_targets(self, player_data):
        """
        Creates injury proxy targets based on performance drops
        """
        
        # Calculates rolling averages
        player_data['actions_7day'] = player_data['total_actions'].rolling(7).mean()
        player_data['actions_30day'] = player_data['total_actions'].rolling(30).mean()
        
        # Performance decline indicators
        player_data['performance_drop'] = (
            player_data['total_actions'] < (player_data['actions_30day'] * 0.7)
        )
        
        # Consecutive games missed (gap analysis)
        player_data['days_since_last_game'] = player_data['game_date'].diff().dt.days
        player_data['extended_absence'] = player_data['days_since_last_game'] > 7
        
        return player_data
    
    def get_modern_era_data(self, start_date='2018-01-01'):
        """
        Focuses on modern NBA era for consistent analysis
        """
        query = f"""
        SELECT COUNT(*) as modern_records
        FROM play_by_play pbp
        JOIN game g ON pbp.game_id = g.game_id  
        WHERE g.game_date >= '{start_date}'
        AND pbp.player1_id IS NOT NULL
        AND pbp.player1_id != '0'
        """
        return pd.read_sql_query(query, self.conn)
    
    def identify_modeling_candidates(self, min_games=50, min_actions_per_game=20):
        """
        Identifies players w/ sufficient data for modeling
        """
        query = f"""
        SELECT 
            pbp.player1_id,
            pbp.player1_name,
            COUNT(DISTINCT pbp.game_id) as games_played,
            COUNT(*) as total_actions,
            CAST(COUNT(*) AS FLOAT) / COUNT(DISTINCT pbp.game_id) as avg_actions_per_game
        FROM play_by_play pbp
        JOIN game g ON pbp.game_id = g.game_id
        WHERE g.game_date >= '2018-01-01'
        AND pbp.player1_id IS NOT NULL
        AND pbp.player1_id != '0'
        GROUP BY pbp.player1_id, pbp.player1_name
        HAVING COUNT(DISTINCT pbp.game_id) >= {min_games}
        AND CAST(COUNT(*) AS FLOAT) / COUNT(DISTINCT pbp.game_id) >= {min_actions_per_game}
        ORDER BY total_actions DESC
        """
        return pd.read_sql_query(query, self.conn)

    def get_modern_analytics_era_data(self, start_date='2015-01-01'):
        """
        Gets data from modern analytics era (2015+) w/ a volume check
        """
        query = f"""
        SELECT 
            COUNT(*) as total_records,
            COUNT(DISTINCT pbp.game_id) as unique_games,
            COUNT(DISTINCT pbp.player1_id) as unique_players,
            MIN(g.game_date) as earliest_date,
            MAX(g.game_date) as latest_date,
            COUNT(DISTINCT SUBSTR(g.game_date, 1, 4)) as seasons_covered
        FROM play_by_play pbp
        JOIN game g ON pbp.game_id = g.game_id
        WHERE g.game_date >= '{start_date}'
        AND pbp.player1_id IS NOT NULL
        AND pbp.player1_id != '0'
        """
        return pd.read_sql_query(query, self.conn)
    
    def get_yearly_breakdown_2015_plus(self):
        """
        Shows data distribution by year from 2015+
        """
        query = """
        SELECT 
            SUBSTR(g.game_date, 1, 4) as year,
            COUNT(*) as records,
            COUNT(DISTINCT pbp.game_id) as games,
            COUNT(DISTINCT pbp.player1_id) as players
        FROM play_by_play pbp
        JOIN game g ON pbp.game_id = g.game_id
        WHERE g.game_date >= '2015-01-01'
        AND pbp.player1_id IS NOT NULL
        AND pbp.player1_id != '0'
        GROUP BY SUBSTR(g.game_date, 1, 4)
        ORDER BY year
        """
        return pd.read_sql_query(query, self.conn)
    
    def analyze_2015_plus_suitability(self):
        """
        Analyzes if 2015+ data is suitable for injury prediction modeling
        """
        
        # Player consistency check
        consistency_query = """
        SELECT 
            pbp.player1_id,
            pbp.player1_name,
            COUNT(DISTINCT SUBSTR(g.game_date, 1, 4)) as seasons_active,
            MIN(g.game_date) as first_game,
            MAX(g.game_date) as last_game,
            COUNT(DISTINCT pbp.game_id) as total_games,
            COUNT(*) as total_actions
        FROM play_by_play pbp
        JOIN game g ON pbp.game_id = g.game_id
        WHERE g.game_date >= '2015-01-01'
        AND pbp.player1_id IS NOT NULL
        AND pbp.player1_id != '0'
        GROUP BY pbp.player1_id, pbp.player1_name
        HAVING COUNT(DISTINCT pbp.game_id) >= 100  -- Players with substantial data
        ORDER BY total_actions DESC
        LIMIT 20
        """
        
        return pd.read_sql_query(consistency_query, self.conn)

    def analyze_missing_games(self, player_id='203507', min_gap_days=7):
        """
        Identifies potential injury periods by finding gaps in game activity
        """
        query = f"""
        WITH player_games AS (
            SELECT DISTINCT
                g.game_date,
                pbp.player1_name
            FROM play_by_play pbp
            JOIN game g ON pbp.game_id = g.game_id
            WHERE pbp.player1_id = '{player_id}'
            AND g.game_date >= '2015-01-01'
            ORDER BY g.game_date
        ),
        game_gaps AS (
            SELECT 
                game_date,
                player1_name,
                LAG(game_date) OVER (ORDER BY game_date) as prev_game,
                julianday(game_date) - julianday(LAG(game_date) OVER (ORDER BY game_date)) as days_gap
            FROM player_games
        )
        SELECT *
        FROM game_gaps 
        WHERE days_gap >= {min_gap_days}
        ORDER BY days_gap DESC
        """
        return pd.read_sql_query(query, self.conn)

    def generate_initial_report(self):
        """
        Generates early data exploration report
        """
        print("NBA Dataset Inital Exploration Report:")
        
        # Table overview
        table_info = self.get_table_info()
        print("\nTable Overview:")
        for table, count in table_info.items():
            print(f"  {table}: {count:,} records")
        
        # Date range analysis
        print("\nDate Range Analysis:")
        date_range, games_per_year = self.get_date_range_analysis()
        print(f"- Date range: {date_range['earliest_date'].iloc[0]} to {date_range['latest_date'].iloc[0]}")
        print(f"- Total games: {date_range['total_games'].iloc[0]:,}")
        print(f"- Total seasons: {date_range['total_seasons'].iloc[0]}")
        
        print("\nRecent years game counts:")
        for _, row in games_per_year.head(5).iterrows():
            print(f"-{row['year']}: {row['games_count']:,} games")
        
        # Performance test
        print("\nPerfromance Test:")
        self.test_play_by_play_join_performance()
        
        # Play-by-play sample analysis
        print("\nPlay-by-Play Sample Analysis:")
        pbp_sample = self.explore_play_by_play_sample(500)  # Smaller sample
        print(f"- Sample size: {len(pbp_sample):,} records")
        print(f"- Date range in sample: {pbp_sample['game_date'].min()} to {pbp_sample['game_date'].max()}")
        print(f"- Unique games in sample: {pbp_sample['game_id'].nunique():,}")
        print(f"- Unique players in sample: {pbp_sample['player1_id'].nunique():,}")
        
        # Events types distribution
        event_counts = pbp_sample['eventmsgtype'].value_counts()
        print(f"- Event types distribution: {dict(event_counts.head())}")
        
        # High usage players (2020+)
        print("\nHigh Usage Players (2020+):")
        high_usage = self.identify_high_usage_players()
        if not high_usage.empty:
            print(high_usage.head(10).to_string(index=False))
        else:
            print("- No high usage players found (may need to lower min_games threshold)")
        
        # Player-game aggregation example
        print("\nPlayer Game Aggregation Example:")
        if not high_usage.empty:
            sample_player_id = high_usage.iloc[0]['player1_id']
            player_games = self.get_player_game_aggregation(sample_player_id, 10)
            print(f"- Sample player: {high_usage.iloc[0]['player1_name']}")
            if not player_games.empty:
                print(player_games.to_string(index=False))
            else:
                print("- No game data found for sample player")
        
        print("\nInital EDA Completed")
        return table_info, pbp_sample, high_usage

In [3]:
explorer = NBADataExplorer()
explorer.connect()

Connected to ..\data\raw\nba.sqlite


# NBA Database Table Schema Analysis

### Key Findings from explorer.explore_table_schemas()

**PLAY_BY_PLAY Table - Core Event Data**
- **34 columns** with granular event tracking
- **Multi-player attribution** (player1, player2, player3) for complex plays
- **Time precision** via pctimestring and wctimestring
- **Event classification** through eventmsgtype and eventmsgactiontype
- **Team context** available for all three player slots
- **Critical for workload analysis** - every player action recorded

**GAME Table: Match Context & Team Stats**
- **55 columns** with comprehensive game level data
- **Home/Away split** for all statistics (doubles most metrics)
- **Box score completeness** - FG, 3PT, FT, rebounds, assists, etc.
- **Game identification** via game_id and game_date 
- **Season context** through season_id and season_type
- **Essential JOIN key** for temporal analysis

**PLAYER Table: Basic Identity**
- **Minimal schema** (5 columns) - basic player roster
- **Simple lookup** for id to full_name mapping
- **Activity status** via is_active flag
- **Limited utility** for feature engineering

**COMMON_PLAYER_INFO Table: Rich Metadata**
- **33 columns** with comprehensive player profiles
- **Physical attributes** - height, weight, birthdate
- **Career span** - from_year, to_year, season_exp
- **Draft information** - draft_year, draft_round, draft_number
- **Position data** for role specific analysis
- **Team affiliation** and status flags
- **Critical for risk profiling** - age, size, experience factors

**Relationship Mapping**
- **play_by_play ↔ game** via game_id (temporal context)
- **play_by_play ↔ common_player_info** via player1_id = person_id
- **Multi-table joins possibly required** for comprehensive player game analysis

**Feature Engineering Implications**
- **Granular workload tracking** possible from play by play events
- **Physical risk factors** available from player info (age, size)
- **Temporal analysis** enabled through game dates
- **Position specific modeling** supported by role classifications
- **Event attribution** handles multi player scenarios

In [4]:
explorer.explore_table_schemas()


PLAY_BY_PLAY Table Schema:
 cid                      name    type  notnull dflt_value  pk
   0                   game_id    TEXT        0       None   0
   1                  eventnum INTEGER        0       None   0
   2              eventmsgtype INTEGER        0       None   0
   3        eventmsgactiontype INTEGER        0       None   0
   4                    period INTEGER        0       None   0
   5              wctimestring    TEXT        0       None   0
   6              pctimestring    TEXT        0       None   0
   7           homedescription    TEXT        0       None   0
   8        neutraldescription    TEXT        0       None   0
   9        visitordescription    TEXT        0       None   0
  10                     score    TEXT        0       None   0
  11               scoremargin    TEXT        0       None   0
  12               person1type    REAL        0       None   0
  13                player1_id    TEXT        0       None   0
  14              player1_n

# NBA Dataset Initial Exploration Report Findings

### Database Scale Assessment

**Table Volume Analysis**
- **Massive play by play dataset**: 13.6M records. The primary data source for modeling
- **Comprehensive game coverage**: 65,698 games across 77 years (1946-2023)
- **Rich player metadata**: 3,632 detailed player profiles
- **Supporting data abundance**: Officials, inactive players, draft history all well-populated
- **Empty table**: team_info_common has 0 records - exclude from pipeline

**Historical Coverage**
- **Full NBA history span**: 1946 to June 2023 (225 seasons)
- **Modern era well represented**: Recent seasons show consistent game counts
- **COVID impact visible**: 2020 season reduced to 756 games vs normal 1,200+
- **Current through playoffs**: Data extends to 2023 Finals completion

### Performance & Feasibility Assessment

**Query Performance Benchmarks**
- **JOIN operations viable**: 24.58 seconds for 320k records indicates scalable architecture  
- **2023+ data subset**: 319,967 records available for recent analysis
- **Processing capacity confirmed**: Large scale feature engineering feasible

**Sample Data Quality**
- **Event distribution patterns**: Rebounds (type 4) most common, followed by missed shots (type 2)
- **Single game depth**: 500 records from one game shows 26 unique players
- **Action variety confirmed**: 5 distinct event types in small sample

### Player Selection for Modeling

**High Usage Player Identification**
- **Giannis Antetokounmpo leads**: 13,706 total actions across 260 games (52.7 per game)
- **Elite player pool identified**: Top 10 players average 40+ actions per game
- **Sufficient modeling candidates**: Multiple players with 200+ games since 2020
- **Data quality issue noted**: 'None' player entries require filtering

**Player Game Aggregation Validation**
- **Aggregation pipeline functional**: Successfully converts play by play to player game stats
- **Feature categories established**: Field goals, rebounds, turnovers, fouls all trackable
- **Temporal context preserved**: Game dates maintained for time series analysis

### Data Pipeline Readiness

**Technical Infrastructure**
- **Database connections stable**: SQLite handling large volumes effectively  
- **JOIN operations optimized**: Complex multi table queries performing adequately
- **Feature engineering foundation**: Player game aggregation methodology proven

**Quality Control Requirements**
- **Null value handling needed**: 'None' player entries must be filtered
- **Data validation required**: Some records show unexpected patterns
- **Modern era focus recommended**: 2015+ data likely optimal for consistency

In [5]:
table_info, pbp_sample, high_usage = explorer.generate_initial_report()

NBA Dataset Inital Exploration Report:

Table Overview:
  common_player_info: 3,632 records
  draft_combine_stats: 1,633 records
  draft_history: 8,257 records
  game: 65,698 records
  game_info: 58,053 records
  game_summary: 58,110 records
  inactive_players: 110,191 records
  line_score: 58,053 records
  officials: 70,971 records
  other_stats: 28,271 records
  play_by_play: 13,592,899 records
  player: 4,815 records
  team: 30 records
  team_details: 27 records
  team_history: 50 records
  team_info_common: 0 records

Date Range Analysis:
- Date range: 1946-11-01 00:00:00 to 2023-06-12 00:00:00
- Total games: 65,698
- Total seasons: 225

Recent years game counts:
-2023: 770 games
-2022: 1,407 games
-2021: 1,693 games
-2020: 756 games
-2019: 1,269 games

Perfromance Test:
Testing play-by-play JOIN performance...
JOIN test completed in 31.31 seconds
- Found 319,967 play-by-play records for 2023+

Play-by-Play Sample Analysis:
- Sample size: 500 records
- Date range in sample: 2020-01

In [6]:
modern_stats = explorer.get_modern_era_data()
print(modern_stats)

   modern_records
0         3017335


In [7]:
clean_sample = explorer.get_clean_player_actions()
print(f"Clean sample size: {len(clean_sample)}")

Clean sample size: 1000


In [8]:
# Test 2000+ data volume
test_2000_query = """
SELECT
    COUNT(*) as total_records,
    COUNT(DISTINCT pbp.game_id) as unique_games,
    COUNT(DISTINCT pbp.player1_id) as unique_players,
    MIN(g.game_date) as earliest_date,
    MAX(g.game_date) as latest_date
FROM play_by_play pbp
JOIN game g ON pbp.game_id = g.game_id
WHERE g.game_date >= '2000-01-01'
AND pbp.player1_id IS NOT NULL
AND pbp.player1_id != '0'
"""

data_2000 = pd.read_sql_query(test_2000_query, explorer.conn)
print("2000+ Era Data After Cleaning:")
for col in data_2000.columns:
    value = data_2000[col].iloc[0]
    if isinstance(value, (int, float)):
        print(f"{col}: {value:,}")
    else:
        print(f"{col}: {value}")

2000+ Era Data After Cleaning:
total_records: 11843898
unique_games: 26493
unique_players: 3716
earliest_date: 2000-01-02 00:00:00
latest_date: 2023-06-09 00:00:00


In [9]:
# Test 2015+ data volume
# - Start of load managmeent
# - Start of tracking cameras in arenas
# - GSW (2015-2016) 73 win season arguably altered pace and style league wide
test_2015_query = """
SELECT
    COUNT(*) as total_records,
    COUNT(DISTINCT pbp.game_id) as unique_games,
    COUNT(DISTINCT pbp.player1_id) as unique_players,
    MIN(g.game_date) as earliest_date,
    MAX(g.game_date) as latest_date
FROM play_by_play pbp
JOIN game g ON pbp.game_id = g.game_id
WHERE g.game_date >= '2015-01-01'
AND pbp.player1_id IS NOT NULL
AND pbp.player1_id != '0'
"""

# Run the fixed query
data_2015 = pd.read_sql_query(test_2015_query, explorer.conn)
print("2015+ Era Data After Cleaning:")
for col in data_2015.columns:
    value = data_2015[col].iloc[0]
    if isinstance(value, (int, float)):
        print(f"{col}: {value:,}")
    else:
        print(f"{col}: {value}")

2015+ Era Data After Cleaning:
total_records: 4782722
unique_games: 10460
unique_players: 2678
earliest_date: 2015-01-01 00:00:00
latest_date: 2023-06-09 00:00:00


# Modern Analytics Era Data Analysis (2015+)

**Overall Volume Assessment**
- **4.78 million play-by-play records** from modern NBA era
- **10,460 unique games** providing comprehensive match coverage
- **2,678 unique players** tracked across 9 seasons
- **Complete temporal span** from January 2015 through June 2023

### Year by Year Data Distribution

**Standard Seasons (2015-2019, 2021-2022)**
- **Consistent volume range**: 540k-590k records per season
- **Stable game counts**: 1,200-1,300 games annually
- **Player pool consistency**: 1,350-1,450 active players per year

### Anomalous Seasons Identified

**2020 Season - COVID Impact**
- **Record reduction**: 317,570 records (45% below normal)
- **Game shortage**: Only 680 games vs typical 1,200+
- **Player impact**: Reduced to 1,289 active players

**2021 Season - Recovery Surge**
- **Peak activity**: 702,405 records (highest in dataset)
- **Extended schedule**: 1,543 games (makeup games included)
- **Normal player participation**: 1,432 players active

**2023 Season - Partial Data**
- **Incomplete season**: 314,238 records through June
- **Playoff-only coverage**: 689 games suggests regular season cutoff
- **Reduced player pool**: 1,096 players (expected for partial season)

### Data Quality for Modeling

**Consistency Assessment**
- **Seven complete seasons** available for training (2015-2019, 2021-2022)
- **Two partial/anomalous seasons** require special handling (2020, 2023)
- **Stable methodology period** captures modern NBA analytics revolution

**Strategic Recommendations**
- **Focus on 2015-2022 data** for primary model training
- **Use 2020 as validation set** to test model robustness during disruptions
- **Reserve 2023 data** for final model evaluation and recency validation

In [10]:
modern_era = explorer.get_modern_analytics_era_data('2015-01-01')
yearly_breakdown = explorer.get_yearly_breakdown_2015_plus()

print("2015+ Analytics Era Data:")
print(modern_era)
print("\nYear By Year Breakdown:")
print(yearly_breakdown)

2015+ Analytics Era Data:
   total_records  unique_games  unique_players        earliest_date  \
0        4782722         10460            2678  2015-01-01 00:00:00   

           latest_date  seasons_covered  
0  2023-06-09 00:00:00                9  

Year By Year Breakdown:
   year  records  games  players
0  2015   582252   1286     1386
1  2016   591428   1302     1355
2  2017   591707   1314     1440
3  2018   551911   1205     1349
4  2019   543627   1162     1349
5  2020   317570    680     1289
6  2021   702405   1543     1432
7  2022   587584   1279     1464
8  2023   314238    689     1096


In [11]:
# Shows top players for injury prediction modeling
top_players_2015_plus = explorer.analyze_2015_plus_suitability()
print("Top Players for Injury Prediction Modeling (2015+):")
print(top_players_2015_plus)

Top Players for Injury Prediction Modeling (2015+):
   player1_id           player1_name  seasons_active           first_game  \
0      203507  Giannis Antetokounmpo               9  2015-01-02 00:00:00   
1      201935           James Harden               9  2015-01-02 00:00:00   
2      201566      Russell Westbrook               9  2015-01-02 00:00:00   
3        2544           LeBron James               9  2015-01-16 00:00:00   
4      203999           Nikola Jokic               9  2015-10-02 00:00:00   
5      201942          DeMar DeRozan               9  2015-01-14 00:00:00   
6      203076          Anthony Davis               9  2015-01-02 00:00:00   
7      201939          Stephen Curry               9  2015-01-02 00:00:00   
8      203081         Damian Lillard               9  2015-01-03 00:00:00   
9      203944          Julius Randle               9  2015-10-04 00:00:00   
10     203083         Andre Drummond               9  2015-01-04 00:00:00   
11     202696         Ni

# NBA Event Types Distribution Analysis (2015+)

### Primary Event Categories for Feature Engineering

**High Volume Events (15%+ of actions)**
- **Event Type 4**: 1.1M occurrences (22.49%) - Rebounds - Core defensive workload metric
- **Event Type 2**: 986k occurrences (20.22%) - Missed shots - Shooting volume indicator  
- **Event Type 1**: 838k occurrences (17.20%) - Made shots - Offensive efficiency marker

**Moderate-Volume Events (5-10% of actions)**
- **Event Type 8**: 480k occurrences (9.85%) - Substitutions - Fatigue/rest management
- **Event Type 3**: 478k occurrences (9.80%) - Free throws - Contact play aftermath
- **Event Type 6**: 435k occurrences (8.93%) - Fouls - Physical contact indicator
- **Event Type 5**: 298k occurrences (6.12%) - Turnovers - Ball security/pressure metric

**Low-Volume Events (Under 3% each)**
- **Event Type 9**: 117k occurrences (2.39%) - Timeouts - Game management
- **Event Types 10, 7, 18**: Under 0.4% each - Technical/administrative events
- **Event Type 11**: 632 occurrences (0.01%) - Rare technical events

### Feature Engineering Implications

**Workload Categories Identified**
- **Shooting Load**: Types 1 + 2 + 3 = 37.4% of all player actions
- **Defensive Load**: Type 4 = 22.5% of actions (rebounds primary)
- **Physical Contact**: Types 6 + 3 = 18.7% (fouls + free throws)
- **Ball Handling**: Type 5 = 6.1% (turnovers indicate possession stress)

**Player Usage Metrics Available**
- **Offensive involvement**: Shot attempts and makes trackable
- **Defensive activity**: Rebounding volume quantifiable  
- **Physical engagement**: Foul participation measurable
- **Game flow impact**: Substitution patterns observable

**Data Quality Validation**
- **95% coverage**: Top 7 event types account for nearly all meaningful player actions
- **Clear categorization**: Events naturally group into workload types
- **Sufficient volume**: All major categories have substantial sample sizes for modeling

In [12]:
# Event types distribution (for feature engineering)
event_analysis_query = """
SELECT 
    eventmsgtype,
    COUNT(*) as event_count,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM play_by_play pbp 
          JOIN game g ON pbp.game_id = g.game_id 
          WHERE g.game_date >= '2015-01-01'), 2) as percentage
FROM play_by_play pbp
JOIN game g ON pbp.game_id = g.game_id
WHERE g.game_date >= '2015-01-01'
AND pbp.player1_id IS NOT NULL
AND pbp.player1_id != '0'
GROUP BY eventmsgtype
ORDER BY event_count DESC
"""

event_types = pd.read_sql_query(event_analysis_query, explorer.conn)
print("Event Types Distribution (2015+):")
print(event_types)

Event Types Distribution (2015+):
    eventmsgtype  event_count  percentage
0              4      1096407       22.49
1              2       985783       20.22
2              1       838354       17.20
3              8       480066        9.85
4              3       477643        9.80
5              6       435195        8.93
6              5       298276        6.12
7              9       116554        2.39
8             10        18112        0.37
9              7        18101        0.37
10            18        17599        0.36
11            11          632        0.01


# Data Quality Assessment (2015+)

**Overall Data Completeness**
- Total play by play records available: 4,875,526
- Records with valid player IDs: 4,782,722 (98.1% completion)
- Records with player names: 4,462,940 (91.5% completion)
- Records with team information: 4,458,150 (91.4% completion)

**Quality Insights**
- Excellent player ID coverage with minimal missing values
- Name and team data gaps likely represent system events or technical timeouts
- 4.7M clean records provide robust foundation for feature engineering
- Data completeness sufficient for reliable injury prediction modeling

**Feature Engineering Implications**
- Filter on player1_id IS NOT NULL for feature engineering
- Strong data quality supports proceeding to workload metric creation
- Missing names/teams won't impact core usage pattern analysis

In [13]:
# Checks for data completeness and quality issues
data_quality_query = """
SELECT 
    'Total Records' as metric,
    COUNT(*) as value
FROM play_by_play pbp
JOIN game g ON pbp.game_id = g.game_id
WHERE g.game_date >= '2015-01-01'

UNION ALL

SELECT 
    'Records with Player ID' as metric,
    COUNT(*) as value
FROM play_by_play pbp
JOIN game g ON pbp.game_id = g.game_id
WHERE g.game_date >= '2015-01-01'
AND pbp.player1_id IS NOT NULL
AND pbp.player1_id != '0'

UNION ALL

SELECT 
    'Records with Player Name' as metric,
    COUNT(*) as value
FROM play_by_play pbp
JOIN game g ON pbp.game_id = g.game_id
WHERE g.game_date >= '2015-01-01'
AND pbp.player1_name IS NOT NULL
AND pbp.player1_name != ''

UNION ALL

SELECT 
    'Records with Team Info' as metric,
    COUNT(*) as value
FROM play_by_play pbp
JOIN game g ON pbp.game_id = g.game_id
WHERE g.game_date >= '2015-01-01'
AND pbp.player1_team_id IS NOT NULL
"""

quality_check = pd.read_sql_query(data_quality_query, explorer.conn)
print("Data Quality Check:")
print(quality_check)

Data Quality Check:
                     metric    value
0             Total Records  4875526
1    Records with Player ID  4782722
2  Records with Player Name  4462940
3    Records with Team Info  4458150


# Giannis Antetokounmpo Game Action Analysis

**Single Game Activity Pattern (April 24, 2023)**
- Total tracked actions in sample: 50 events across all 4 periods
- Event type breakdown: Field goals (1), missed shots (2), rebounds (4), fouls (6), substitutions (8), turnovers (5)
- Game flow: Consistent involvement from Q1 through Q4 with substitution patterns

**Key Insights for Feature Engineering**
- High activity density: Multiple events per minute of game time
- Diverse action types: Scoring, rebounding, defensive plays, turnovers all captured
- Detailed descriptions provide context for shot difficulty and play types
- Clear substitution tracking shows rest periods and load management

**Workload Categorization Opportunities**
- Physical contact events: Driving layups, putback dunks, offensive fouls
- High intensity actions: Fast break plays, driving attempts
- Skill based events: 3-point attempts, free throws, assists
- Defensive load: Rebounds, personal fouls, blocks

**Data Quality Observations**
- Descriptive text captures play context and difficulty
- Time stamps enable precise game flow analysis
- Multiple description fields provide comprehensive event details
- Perfect foundation for creating nuanced workload metrics

In [14]:
# Deep dive into Giannis (most actions compared to rest) to understand data structure
giannis_sample_query = """
SELECT 
    game_date,
    eventmsgtype,
    eventmsgactiontype,
    period,
    pctimestring,
    homedescription,
    visitordescription,
    neutraldescription
FROM play_by_play pbp
JOIN game g ON pbp.game_id = g.game_id
WHERE g.game_date >= '2022-01-01'
AND pbp.player1_id = '203507'  -- Giannis
ORDER BY game_date DESC, eventnum
LIMIT 50
"""

giannis_sample = pd.read_sql_query(giannis_sample_query, explorer.conn)
print("Sample of Giannis's Actions (Recent Games):")
print(giannis_sample)

Sample of Giannis's Actions (Recent Games):
              game_date  eventmsgtype  eventmsgactiontype  period  \
0   2023-04-24 00:00:00             1                  41       1   
1   2023-04-24 00:00:00             4                   0       1   
2   2023-04-24 00:00:00             1                   9       1   
3   2023-04-24 00:00:00             2                  79       1   
4   2023-04-24 00:00:00             1                   6       1   
5   2023-04-24 00:00:00             3                  10       1   
6   2023-04-24 00:00:00             2                   3       1   
7   2023-04-24 00:00:00             2                  63       1   
8   2023-04-24 00:00:00             8                   0       1   
9   2023-04-24 00:00:00             6                   1       1   
10  2023-04-24 00:00:00             4                   0       1   
11  2023-04-24 00:00:00             2                  43       1   
12  2023-04-24 00:00:00             4                   0  

# Player Absence Pattern Analysis

**Giannis Antetokounmpo - Game Gap Analysis**
- Total gaps of 7+ days identified: 37 instances
- Offseason breaks dominate: 144-173 day gaps between seasons (normal)
- Mid-season absences: 8-13 day gaps indicate rest/minor injury management
- Notable pattern: Consistent 7-day gaps suggest scheduled rest periods

### Cross Player Injury Pattern Comparison

**James Harden**
- Extended absences (14+ days): 14 instances
- Longest gap: 169 days (offseason)
- Pattern: Primarily offseason with minimal mid season extended absences

**Russell Westbrook** 
- Extended absences (14+ days): 13 instances
- Longest gap: 186 days (offseason)
- Similar pattern to Harden with mostly seasonal breaks

**LeBron James**
- Extended absences (14+ days): 15 instances  
- Longest gap: 210 days (2019 offseason)
- Notable: 142-day COVID suspension gap in 2020

**Joel Embiid - High Injury Risk Profile**
- Extended absences (14+ days): 13 instances
- **Critical finding**: 257-day gap in 2017 (major injury)
- Multiple 160+ day gaps suggest recurring injury issues
- Clear injury prone pattern compared to other stars

### Feature Engineering Implications

- 14+ day gaps effectively identify injury periods vs rest
- Embiid's extreme gaps validate injury proxy approach
- Seasonal patterns (140-210 days) easily filtered out
- 7-13 day gaps capture load management strategies

In [15]:
# Missing Games Analysis (Analyzes Giannis (highest activity player))
print("Giannis Antetokounmpo - Potential Injury/Rest Periods:")
giannis_gaps = explorer.analyze_missing_games('203507', min_gap_days=7)
print(giannis_gaps)

# Analyzes a few more top players for comparison
top_players = [
    ('201935', 'James Harden'),
    ('201566', 'Russell Westbrook'), 
    ('2544', 'LeBron James'),
    ('203954', 'Joel Embiid')  # INJURRRRRY PRONEEEEEEEE
]

for player_id, player_name in top_players:
    print(f"\n{player_name} - Gaps of 14+ days:")
    gaps = explorer.analyze_missing_games(player_id, min_gap_days=14)
    if len(gaps) > 0:
        print(f"- Found {len(gaps)} extended absences")
        print(gaps[['game_date', 'prev_game', 'days_gap']].head(3))
    else:
        print("- No extended absences found")
    print()

Giannis Antetokounmpo - Potential Injury/Rest Periods:
              game_date           player1_name            prev_game  days_gap
0   2016-10-03 00:00:00  Giannis Antetokounmpo  2016-04-13 00:00:00     173.0
1   2018-10-17 00:00:00  Giannis Antetokounmpo  2018-04-28 00:00:00     172.0
2   2017-10-06 00:00:00  Giannis Antetokounmpo  2017-04-24 00:00:00     165.0
3   2015-10-06 00:00:00  Giannis Antetokounmpo  2015-04-30 00:00:00     159.0
4   2019-10-26 00:00:00  Giannis Antetokounmpo  2019-05-25 00:00:00     154.0
5   2020-07-31 00:00:00  Giannis Antetokounmpo  2020-03-06 00:00:00     147.0
6   2022-10-06 00:00:00  Giannis Antetokounmpo  2022-05-15 00:00:00     144.0
7   2020-12-12 00:00:00  Giannis Antetokounmpo  2020-09-06 00:00:00      97.0
8   2021-10-10 00:00:00  Giannis Antetokounmpo  2021-07-20 00:00:00      82.0
9   2021-04-15 00:00:00  Giannis Antetokounmpo  2021-04-02 00:00:00      13.0
10  2021-12-25 00:00:00  Giannis Antetokounmpo  2021-12-13 00:00:00      12.0
11  2023-